### _LCEL - LANGCHAIN EXPRESSION LANGUAGE_ ###

In [15]:
#Import libraries
import os
from dotenv import load_dotenv
load_dotenv()

True

In [14]:
#Setting environmental variables
import openai
openai.api_key = os.getenv('OPENAI_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')


In [24]:
from langchain_groq import ChatGroq
model = ChatGroq(model ='gemma2-9b-it',groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7effdce3ca30>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7effdce3da80>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [27]:
messages = [
    ("system", "You are a helpful translator. Translate the user sentence to hindi."),
    ("human", "I love programming."),
]
response = model.invoke(messages)
print(response.content)

मुझे प्रोग्रामिंग बहुत पसंद है।  (Mujhe programamming bahut pasand hai.) 

